## 事前準備

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/nlp100_2025/ja/

import os

current_directory = os.getcwd()
print(f"The Current Directory: {current_directory}")

# 第8章: ニューラルネット

第7章で取り組んだポジネガ分類を題材として、ニューラルネットワークで分類モデルを実装する。なお、この章ではPyTorchやTensorFlow、JAXなどの深層学習フレームワークを活用せよ。

## 70. 単語埋め込みの読み込み

事前学習済み単語埋め込みを活用し、$|V| \times d_\rm{emb}$ の単語埋め込み行列$\pmb{E}$を作成せよ。ここで、$|V|$は単語埋め込みの語彙数、$d_\rm{emb}$は単語埋め込みの次元数である。ただし、単語埋め込み行列の先頭の行ベクトル$\pmb{E}_{0,:}$は、将来的にパディング（`<PAD>`）トークンの埋め込みベクトルとして用いたいので、ゼロベクトルとして予約せよ。ゆえに、$\pmb{E}$の2行目以降に事前学習済み単語埋め込みを読み込むことになる。

もし、Google Newsデータセットの[学習済み単語ベクトル](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing)（300万単語・フレーズ、300次元）を全て読み込んだ場合、$|V|=3000001, d_\rm{emb}=300$になるはずである（ただ、300万単語の中には、殆ど用いられない稀な単語も含まれるので、語彙を削減した方がメモリの節約になる）。

また、単語埋め込み行列の構築と同時に、単語埋め込み行列の各行のインデックス番号（トークンID）と、単語（トークン）への双方向の対応付けを保持せよ。

In [ ]:
!uv pip install gensim==4.3.0
!uv pip install numpy==1.26.4
!uv pip install scipy==1.12.0

In [ ]:
from gensim.models import KeyedVectors

model_path = '../data/GoogleNews-vectors-negative300.bin.gz'

try:
    # モデルの読み込み (時間がかかることがあります)
    print("単語ベクトルモデルを読み込んでいます... (数分かかる場合があります)")
    word_vectors = KeyedVectors.load_word2vec_format(model_path, binary=True)
    print("モデルの読み込みが完了しました。")

    # "United States" (内部表現 "United_States") の単語ベクトルを取得
    target_word = "United_States"

    if target_word in word_vectors:
        vector_united_states = word_vectors[target_word]
        print(f"\n単語 '{target_word}' のベクトル:")
        print(vector_united_states)
        print(f"\nベクトルの次元数: {len(vector_united_states)}")
    else:
        print(f"エラー: 単語 '{target_word}' はボキャブラリ内に見つかりませんでした。")
        print(f"代わりに 'United States' で試してみます...")
        target_word_alt = "United States" # スペース区切りも試す (通常は _ 区切り)
        if target_word_alt in word_vectors:
            vector_united_states = word_vectors[target_word_alt]
            print(f"\n単語 '{target_word_alt}' のベクトル:")
            print(vector_united_states)
            print(f"\nベクトルの次元数: {len(vector_united_states)}")
        else:
            print(f"エラー: 単語 '{target_word_alt}' もボキャブラリ内に見つかりませんでした。")


except FileNotFoundError:
    print(f"エラー: 指定されたパスにファイルが見つかりません: {model_path}")
    print("Google Newsの単語ベクトルファイルをダウンロードし、正しいパスを指定してください。")
except Exception as e:
    print(f"モデルの読み込み中またはベクトル取得中にエラーが発生しました: {e}")

In [ ]:
import numpy as np
from gensim.models import KeyedVectors

# word_vectors は問題50で読み込まれた KeyedVectors オブジェクトとします。
# もし word_vectors が未定義の場合は、問題50のコードを先に実行してモデルをロードしてください。

if 'word_vectors' not in locals() or word_vectors is None:
    print("エラー: 単語ベクトルモデル 'word_vectors' が読み込まれていません。")
    print("問題50のコードを実行して、先にモデルをロードしてください。")
    # この後の処理に進めないため、ここで処理を中断
    # raise NameError("word_vectors is not defined")
else:
    print("読み込み済みの単語ベクトルモデルを使用します。")

    # 単語ベクトルの次元数を取得
    embedding_dim = word_vectors.vector_size
    print(f"単語ベクトルの次元数 (d_emb): {embedding_dim}")

    # 語彙リストとIDマッピングの初期化
    # <PAD> トークンを追加
    PAD_TOKEN = "<PAD>"
    PAD_ID = 0
    word_to_id = {PAD_TOKEN: 0}
    id_to_word = {0: PAD_TOKEN}

    # 語彙リスト (gensimモデルのキーをそのまま使う。順序も保持される)
    # word_vectors.index_to_key で単語リストを取得可能
    vocabulary_words = word_vectors.index_to_key

    # 埋め込み行列の語彙数 (|V|) は、元の語彙数 + 1 (<PAD>トークン分)
    vocab_size = len(vocabulary_words) + 1
    print(f"元モデルの語彙数: {len(vocabulary_words)}")
    print(f"パディングトークンを含む総語彙数 (|V|): {vocab_size}")

    # 単語埋め込み行列Eを初期化 (0行目はパディング用にゼロベクトル、残りは実際のベクトル)
    # 0行目は既にゼロで初期化される
    embedding_matrix = np.zeros((vocab_size, embedding_dim), dtype=np.float32)

    print(f"単語埋め込み行列 E の形状: {embedding_matrix.shape}")

    # word_vectorsから単語とベクトルを読み込み、word_to_id, id_to_word, embedding_matrix を構築
    # ID=0 は <PAD> なので、実際の単語は ID=1 から開始
    for i, word in enumerate(vocabulary_words):
        token_id = i + 1 # IDは1から開始
        word_to_id[word] = token_id
        id_to_word[token_id] = word
        embedding_matrix[token_id] = word_vectors[word]

    print("\n単語埋め込み行列とIDマッピングの構築が完了しました。")

    # 簡単なテスト
    print(f"\n<PAD> のID: {word_to_id[PAD_TOKEN]}")
    print(f"<PAD> のベクトル (embedding_matrix[0]): {embedding_matrix[word_to_id[PAD_TOKEN]][:5]}... (最初の5次元)") # 全て0のはず

    # 何か適当な単語でテスト
    sample_word = "king" # word_vectors に含まれる単語を選ぶ
    if sample_word in word_to_id:
        sample_word_id = word_to_id[sample_word]
        print(f"\n単語 '{sample_word}' のID: {sample_word_id}")
        print(f"ID {sample_word_id} に対応する単語: {id_to_word[sample_word_id]}")
        print(f"'{sample_word}' のベクトル (word_vectors['{sample_word}'][:5]): {word_vectors[sample_word][:5]}...")
        print(f"'{sample_word}' のベクトル (embedding_matrix[{sample_word_id}][:5]): {embedding_matrix[sample_word_id][:5]}...")
        # 上の2つのベクトルが一致することを確認
        if np.allclose(word_vectors[sample_word], embedding_matrix[sample_word_id]):
            print(f"'{sample_word}' のベクトルは正しくコピーされました。")
        else:
            print(f"警告: '{sample_word}' のベクトルが正しくコピーされていません。")
    else:
        print(f"テスト単語 '{sample_word}' は語彙に含まれていません。")

## 71. データセットの読み込み

[General Language Understanding Evaluation (GLUE)](https://gluebenchmark.com/) ベンチマークで配布されている[Stanford Sentiment Treebank (SST)](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip) をダウンロードし、訓練セット（train.tsv）と開発セット（dev.tsv）のテキストと極性ラベルと読み込み、全てのテキストをトークンID列に変換せよ。このとき、単語埋め込みの語彙でカバーされていない単語は無視し、トークン列に含めないことにせよ。また、テキストの全トークンが単語埋め込みの語彙に含まれておらず、空のトークン列となってしまう事例は、訓練セットおよび開発セットから削除せよ（このため、第7章の実験で得られた正解率と比較できなくなることに注意せよ）。

事例の表現方法は任意でよいが、例えば"contains no wit , only labored gags"がネガティブに分類される事例は、次のような辞書オブジェクトで表現すればよい。

```
{'text': 'contains no wit , only labored gags',
 'label': tensor([0.]),
 'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])}
```

この例では、`text`はテキスト、`label`は分類ラベル（ポジティブなら`tensor([1.])`、ネガティブなら`tensor([0.])`）、`input_ids`はテキストのトークン列をID列で表現している。

In [ ]:
!uv pip install torch

In [ ]:
import pandas as pd
import torch # PyTorchを使用
import os

# --- 前提となる変数 (問題70および問題60から) ---
# word_to_id: 単語からトークンIDへの辞書 (問題70で作成済みとします)
# train_file_path: train.tsvへのパス (問題60で定義済みとします)
# dev_file_path: dev.tsvへのパス (問題60で定義済みとします)

# もし word_to_id やファイルパスが現在のセッションにない場合は、
# 問題70や問題60を再実行して準備してください。
# 例:
# word_to_id = {PAD_TOKEN: 0, 'word1': 1, ...} # 問題70の word_to_id
# train_file_path = "../data/SST-2_data/SST-2/train.tsv" # 問題60のパス
# dev_file_path = "../data/SST-2_data/SST-2/dev.tsv"   # 問題60のパス

if 'word_to_id' not in locals():
    print("エラー: 'word_to_id' が定義されていません。問題70を先に実行してください。")
    # この後の処理に進めないため、ここで処理を中断
    raise NameError("word_to_id is not defined.")

# 問題60でSST-2データを展開した親ディレクトリへのパスを想定
base_data_dir_ch7 = '../data/SST-2_data'
train_file_path = os.path.join(base_data_dir_ch7, "SST-2/train.tsv")
dev_file_path = os.path.join(base_data_dir_ch7, "SST-2/dev.tsv")

# --- ここから問題71の処理 ---
def text_to_token_ids(text, word_to_id_map):
    """テキストを単語に分割し、word_to_id_map を使ってトークンIDのリストに変換する。
       語彙にない単語は無視する。
    """
    if not isinstance(text, str): # 万が一テキストが文字列でない場合
        text = str(text)

    tokens = text.split(' ') # スペースで分割 (問題61と同様)
    token_ids = [word_to_id_map[word] for word in tokens if word in word_to_id_map]
    return token_ids

def load_and_process_sst2_dataset(file_path, word_to_id_map, dataset_name="データセット"):
    """SST-2データセットのTSVファイルを読み込み、指定の形式に処理する。"""
    processed_data = []
    skipped_empty_count = 0

    try:
        df = pd.read_csv(file_path, sep='\t')
        print(f"\n--- {dataset_name} ({os.path.basename(file_path)}) ---")
        print(f"元の事例数: {len(df)}")

        for index, row in df.iterrows():
            text = str(row['sentence'])
            label = int(row['label']) # ラベルを整数 (0 or 1) に

            token_ids = text_to_token_ids(text, word_to_id_map)

            # トークンID列が空になった事例は削除 (無視)
            if not token_ids:
                skipped_empty_count += 1
                continue

            processed_data.append({
                'text': text,
                'label': torch.tensor([float(label)]), # ラベルをfloatのテンソルに
                'input_ids': torch.tensor(token_ids, dtype=torch.long) # トークンID列をlongのテンソルに
            })

        print(f"語彙外単語無視後の処理済み事例数: {len(processed_data)}")
        print(f"空のトークン列のため削除された事例数: {skipped_empty_count}")
        return processed_data

    except FileNotFoundError:
        print(f"エラー: ファイル '{file_path}' が見つかりません。")
        return None
    except KeyError:
        print(f"エラー: ファイル '{file_path}' に必要な列 ('sentence' or 'label') が見つかりません。")
        return None
    except Exception as e:
        print(f"ファイル '{file_path}' の処理中にエラーが発生しました: {e}")
        return None

# word_to_id が問題70で正しく作成されていることを前提とする

print("訓練データの処理を開始します...")
train_dataset_processed = load_and_process_sst2_dataset(train_file_path, word_to_id, "訓練データ")

print("\n開発（検証）データの処理を開始します...")
dev_dataset_processed = load_and_process_sst2_dataset(dev_file_path, word_to_id, "開発データ")

# 処理結果の最初の数件を表示して確認
if train_dataset_processed:
    print("\n訓練データの最初の3事例（処理後）:")
    for i in range(min(3, len(train_dataset_processed))):
        print(train_dataset_processed[i])

if dev_dataset_processed:
    print("\n開発データの最初の3事例（処理後）:")
    for i in range(min(3, len(dev_dataset_processed))):
        print(dev_dataset_processed[i])

# これで train_dataset_processed と dev_dataset_processed に
# 処理済みのデータが格納されました。
# これらは後の問題で使用します。

## 72. Bag of wordsモデルの構築

単語埋め込みの平均ベクトルでテキストの特徴ベクトルを表現し、重みベクトルとの内積でポジティブ及びネガティブを分類するニューラルネットワーク（ロジスティック回帰モデル）を設計せよ。

In [ ]:
import torch
import torch.nn as nn
import numpy as np # embedding_matrix を PyTorchテンソルに変換するために使用

# --- 前提となる変数 (問題70から) ---
# embedding_matrix: 単語埋め込み行列 (NumPy配列)
# vocab_size: 総語彙数
# embedding_dim: 単語ベクトルの次元数
# PAD_ID = 0 (パディングトークンのID)

# もし存在しない場合は、問題70を再実行して準備
if 'embedding_matrix' not in locals() or \
   'vocab_size' not in locals() or \
   'embedding_dim' not in locals() or \
   'PAD_ID' not in locals(): # PAD_ID は直接コード内で0として使われることもあるが、念のため
    print("エラー: 'embedding_matrix', 'vocab_size', 'embedding_dim', または 'PAD_ID' が定義されていません。")
    print("問題70を先に実行して、これらの変数を準備してください。")
    raise NameError("Required variables from Problem 70 are not defined.")

class SimpleBOWClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, pretrained_embeddings, padding_idx):
        super(SimpleBOWClassifier, self).__init__()

        # 1. 単語埋め込み層
        # 事前学習済み重みをロードし、学習中は更新しない (freeze)
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.embedding.weight = nn.Parameter(pretrained_embeddings, requires_grad=False)

        # 2. 線形層 (平均化された埋め込みベクトルを入力とし、1つの値を出力)
        # 出力はロジット (シグモイド関数を適用する前の値) とする
        self.fc = nn.Linear(embedding_dim, 1)

    def forward(self, input_ids):
        # input_ids: (バッチサイズ, シーケンス長) のテンソル

        # 埋め込みベクトルを取得
        # embedded: (バッチサイズ, シーケンス長, embedding_dim)
        embedded = self.embedding(input_ids)

        # パディングを考慮した平均プーリング
        # マスクを作成 (パディング部分は0、非パディング部分は1)
        # input_idsがpadding_idxでない箇所が1、padding_idxの箇所が0になるマスク
        mask = (input_ids != self.embedding.padding_idx).unsqueeze(-1).float()
        # mask: (バッチサイズ, シーケンス長, 1)

        # マスクを適用してパディング部分のベクトルをゼロにする (元々ゼロベクトルだが念のため)
        # embedded = embedded * mask # embedding_matrixのpadding_idx行がゼロなら不要な場合も

        # 各シーケンスの実際の長さ（非パディングトークンの数）を計算
        # lengths: (バッチサイズ)
        lengths = mask.sum(dim=1)
        lengths = lengths.clamp(min=1) # ゼロ除算を避けるため、最小値を1に（全てパディングの稀なケース対策）

        # マスクされた埋め込みベクトルの合計を計算
        # sum_embedded: (バッチサイズ, embedding_dim)
        sum_embedded = (embedded * mask).sum(dim=1) # パディング部分を除外して合計

        # 平均を計算
        # mean_embedded: (バッチサイズ, embedding_dim)
        mean_embedded = sum_embedded / lengths

        # 線形層に入力
        # logits: (バッチサイズ, 1)
        logits = self.fc(mean_embedded)

        return logits

# --- モデルのインスタンス化と簡単なテスト ---
# PyTorchテンソルに変換
embedding_tensor = torch.tensor(embedding_matrix, dtype=torch.float)

# モデルのインスタンスを作成
model_bow = SimpleBOWClassifier(vocab_size, embedding_dim, embedding_tensor, PAD_ID)

# モデルの構造を表示して確認
print("設計したBag of Wordsモデルの構造:")
print(model_bow)

# 簡単なダミー入力でフォワードパスをテスト
# バッチサイズ2, シーケンス長5 のダミー入力 (トークンID)
dummy_input_ids = torch.tensor([[10, 20, 30, 0, 0],  # 最初の事例は3トークン + 2パディング
                                [40, 50,  0, 0, 0]], # 2番目の事例は2トークン + 3パディング
                               dtype=torch.long)

# モデルが学習モードか評価モードか (dropoutなどがある場合は影響)
# model_bow.eval() # 評価時
# model_bow.train() # 学習時

if dummy_input_ids.max().item() < vocab_size: # ダミーIDが語彙サイズ内か確認
    try:
        print("\nダミー入力でフォワードパスをテストします...")
        dummy_output = model_bow(dummy_input_ids)
        print("ダミー出力 (ロジット):")
        print(dummy_output)
        print(f"出力形状: {dummy_output.shape}") # 期待: (バッチサイズ, 1)
    except Exception as e:
        print(f"ダミー入力でのフォワードパステスト中にエラー: {e}")
else:
    print("\nダミー入力のIDが語彙サイズを超えています。テストをスキップします。")
    print(f"ダミー入力の最大ID: {dummy_input_ids.max().item()}, 語彙サイズ: {vocab_size}")

# この model_bow が問題72で設計したモデルとなります。
# 次の問題73で、このモデルの学習を行います。

## 73. モデルの学習

問題72で設計したモデルの重みベクトルを訓練セット上で学習せよ。ただし、学習中は単語埋め込み行列の値を固定せよ（単語埋め込み行列のファインチューニングは行わない）。また、学習時に損失値を表示するなど、学習の進捗状況をモニタリングできるようにせよ。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset # DataLoaderを使用する場合

# --- 前提となる変数 ---
# model_bow: 問題72で作成した SimpleBOWClassifier のインスタンス
# train_dataset_processed: 問題71で作成した訓練データのリスト
# dev_dataset_processed: 問題71で作成した開発データのリスト (エポックごとの評価用)

# これらの変数が現在のセッションに存在することを確認してください。
if 'model_bow' not in locals() or \
   'train_dataset_processed' not in locals() or not train_dataset_processed:
    print("エラー: 'model_bow' または 'train_dataset_processed' が定義されていないか、データが空です。")
    print("問題71および72を先に実行して、これらの変数を準備してください。")
    raise NameError("Required variables/data not defined or empty.")

# PyTorchのDatasetクラスを作成 (DataLoaderで使いやすくするため)
class SentimentDataset(Dataset):
    def __init__(self, processed_data):
        self.data = processed_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]['input_ids'], self.data[idx]['label']

# データセットとデータローダーの準備
# ここではバッチサイズ1で、各事例を個別に処理する形をまず示します。
# 問題75, 76で本格的なパディングとミニバッチを導入します。
batch_size = 1 # まずは1で（問題76で本格的なバッチ処理）

train_torch_dataset = SentimentDataset(train_dataset_processed)
train_dataloader = DataLoader(train_torch_dataset, batch_size=batch_size, shuffle=True)

# (オプション) 検証用データローダー
if 'dev_dataset_processed' in locals() and dev_dataset_processed:
    dev_torch_dataset = SentimentDataset(dev_dataset_processed)
    dev_dataloader = DataLoader(dev_torch_dataset, batch_size=batch_size) # シャッフルは不要

# 学習パラメータ
learning_rate = 1e-3
num_epochs = 5 # 学習エポック数 (適宜調整)

# 損失関数と最適化アルゴリズム
# モデルの出力がロジットなので BCEWithLogitsLoss を使用
criterion = nn.BCEWithLogitsLoss()
# 最適化対象はモデルの全パラメータのうち requires_grad=True のもの
# SimpleBOWClassifierでは埋め込み層の重みは requires_grad=False に設定済み
optimizer = optim.Adam(model_bow.parameters(), lr=learning_rate)
# もしfc層のパラメータのみを更新対象とするなら:
# optimizer = optim.Adam(model_bow.fc.parameters(), lr=learning_rate)


print("モデルの学習を開始します...")
# --- 学習ループ ---
for epoch in range(num_epochs):
    model_bow.train() # モデルを学習モードに設定

    running_loss = 0.0
    num_processed_samples = 0

    for i, (input_ids_batch, labels_batch) in enumerate(train_dataloader):
        # input_ids_batch: (batch_size, seq_len)
        # labels_batch: (batch_size, 1)

        # 勾配を初期化
        optimizer.zero_grad()

        # 順伝播
        outputs = model_bow(input_ids_batch) # outputs: (batch_size, 1)

        # 損失計算
        loss = criterion(outputs, labels_batch) # labels_batchもoutputsと同じ形状・型である必要がある

        # 誤差逆伝播
        loss.backward()

        # パラメータ更新
        optimizer.step()

        running_loss += loss.item() * input_ids_batch.size(0)
        num_processed_samples += input_ids_batch.size(0)

        # 学習の進捗を表示 (例: 1000バッチごと)
        if (i + 1) % 1000 == 0 or batch_size * (i + 1) >= len(train_torch_dataset) :
            avg_loss_so_far = running_loss / num_processed_samples
            print(f"  Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(train_dataloader)}], Avg Loss: {avg_loss_so_far:.4f}")

    epoch_loss = running_loss / len(train_torch_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}] 完了, 平均損失: {epoch_loss:.4f}")

    # (オプション) 各エポックの終わりに検証データで性能を軽く評価
    if 'dev_dataloader' in locals():
        model_bow.eval() # モデルを評価モードに
        correct_dev = 0
        total_dev = 0
        with torch.no_grad(): # 勾配計算をしない
            for input_ids_dev, labels_dev in dev_dataloader:
                outputs_dev = model_bow(input_ids_dev)
                predicted_probs = torch.sigmoid(outputs_dev) # ロジットを確率に変換
                predicted_labels = (predicted_probs > 0.5).float() # 0.5を閾値として0 or 1に
                total_dev += labels_dev.size(0)
                correct_dev += (predicted_labels == labels_dev).sum().item()
        dev_accuracy = 100 * correct_dev / total_dev if total_dev > 0 else 0
        print(f"  Epoch [{epoch+1}/{num_epochs}], 検証データ正解率: {dev_accuracy:.2f}%")


print("\nモデルの学習が完了しました。")
# 学習済みの model_bow は次の問題74で使用します。

## 74. モデルの評価

問題73で学習したモデルの開発セットにおける正解率を求めよ。

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset # DataLoaderを再度使う場合

# --- 前提となる変数 ---
# model_bow: 問題73で学習済みの SimpleBOWClassifier のインスタンス
# dev_dataset_processed: 問題71で作成した開発データのリスト
# SentimentDataset: 問題73で定義したカスタムDatasetクラス (DataLoaderを使う場合)

# これらの変数が現在のセッションに存在することを確認してください。
if 'model_bow' not in locals() or \
   'dev_dataset_processed' not in locals() or not dev_dataset_processed:
    print("エラー: 'model_bow' または 'dev_dataset_processed' が定義されていないか、データが空です。")
    print("問題71および73を先に実行して、これらの変数とデータを準備してください。")
    raise NameError("Required variables/data not defined or empty.")

if 'SentimentDataset' not in locals(): # もし SentimentDatasetクラスが未定義なら問題73からコピー
    class SentimentDataset(Dataset):
        def __init__(self, processed_data):
            self.data = processed_data
        def __len__(self):
            return len(self.data)
        def __getitem__(self, idx):
            return self.data[idx]['input_ids'], self.data[idx]['label']

# 開発（検証）データローダーの準備 (問題73で作成していればそれを再利用可)
# ここでは改めて作成する例を示します。バッチサイズは評価時には大きくても問題ないことが多いです。
dev_batch_size = 1 # 評価時のバッチサイズ (適宜調整)
dev_torch_dataset_eval = SentimentDataset(dev_dataset_processed)
dev_dataloader_eval = DataLoader(dev_torch_dataset_eval, batch_size=dev_batch_size)

print("\n学習済みモデルを開発データで評価します...")

# モデルを評価モードに設定
model_bow.eval()

total_correct_dev = 0
total_samples_dev = 0

# 勾配計算を無効にするコンテキスト
with torch.no_grad():
    for input_ids_batch, labels_batch in dev_dataloader_eval:
        # input_ids_batch: (batch_size, seq_len)
        # labels_batch: (batch_size, 1)

        # 順伝播
        outputs = model_bow(input_ids_batch) # outputs: (batch_size, 1) - ロジット

        # ロジットを確率に変換 (0から1の範囲)
        predicted_probs = torch.sigmoid(outputs)

        # 確率を0.5を閾値として0または1のラベルに変換
        predicted_labels = (predicted_probs > 0.5).float() # .float() でラベルと同じ型に

        # 正解数をカウント
        total_correct_dev += (predicted_labels == labels_batch).sum().item()
        total_samples_dev += labels_batch.size(0)

# 正解率の計算
if total_samples_dev > 0:
    accuracy_dev = 100 * total_correct_dev / total_samples_dev
    print(f"\n開発データにおける正解率: {accuracy_dev:.2f}%")
    print(f"  正解した事例数: {total_correct_dev}")
    print(f"  総事例数: {total_samples_dev}")
else:
    print("開発データが空であるか、処理できませんでした。")

## 75. パディング

複数の事例が与えられたとき、これらをまとめて一つのテンソル・オブジェクトで表現する関数`collate`を実装せよ。与えられた複数の事例のトークン列の長さが異なるときは、トークン列の長さが最も長いものに揃え、0番のトークンIDでパディングをせよ。さらに、トークン列の長さが長いものから順に、事例を並び替えよ。

例えば、訓練データセットの冒頭の4事例が次のように表されているとき、

```
[{'text': 'hide new secretions from the parental units',
  'label': tensor([0.]),
  'input_ids': tensor([  5785,     66, 113845,     18,     12,  15095,   1594])},
 {'text': 'contains no wit , only labored gags',
  'label': tensor([0.]),
  'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])},
 {'text': 'that loves its characters and communicates something rather beautiful about human nature',
  'label': tensor([1.]),
  'input_ids': tensor([    4,  5053,    45,  3305, 31647,   348,   904,  2815,    47,  1276,  1964])},
 {'text': 'remains utterly satisfied to remain the same throughout',
  'label': tensor([0.]),
  'input_ids': tensor([  987, 14528,  4941,   873,    12,   208,   898])}]
```

`collate`関数を通した結果は以下のようになることが想定される。

```
{'input_ids': tensor([
    [     4,   5053,     45,   3305,  31647,    348,    904,   2815,     47,   1276,   1964],
    [  5785,     66, 113845,     18,     12,  15095,   1594,      0,      0,      0,      0],
    [   987,  14528,   4941,    873,     12,    208,    898,      0,      0,      0,      0],
    [  3475,     87,  15888,     90,  27695,  42637,      0,      0,      0,      0,      0]]),
 'label': tensor([
    [1.],
    [0.],
    [0.],
    [0.]])}
```


In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence # パディングに便利
from torch.utils.data import DataLoader, Dataset # DataLoaderのテスト用

# --- 前提となる変数・クラス ---
# PAD_ID = 0 (問題70で定義)
# SentimentDataset クラス (問題73または74で定義)
# train_dataset_processed (問題71で作成)

# これらの変数が現在のセッションに存在することを確認してください。
if 'PAD_ID' not in locals():
    print("エラー: 'PAD_ID' が定義されていません。問題70を先に実行してください。")
    raise NameError("PAD_ID is not defined.")
if 'SentimentDataset' not in locals():
    print("エラー: 'SentimentDataset' クラスが定義されていません。問題73または74のコードを確認してください。")
    # もし未定義なら、ここで再度定義するか、該当セルを実行
    class SentimentDataset(Dataset): # 再定義の例
        def __init__(self, processed_data):
            self.data = processed_data
        def __len__(self):
            return len(self.data)
        def __getitem__(self, idx):
            # 各要素は {'input_ids': tensor, 'label': tensor} の辞書
            # collate_fn が (input_ids, label) のタプルのリストを期待する場合
            return self.data[idx]['input_ids'], self.data[idx]['label']
if 'train_dataset_processed' not in locals() or not train_dataset_processed:
    print("エラー: 'train_dataset_processed' が定義されていないか空です。問題71を実行してください。")
    raise NameError("train_dataset_processed is not defined or empty.")

# --- ここから問題75の collate 関数の実装 ---

def collate_fn_custom_pad_sort(batch, padding_value=PAD_ID):
    """
    バッチ内の事例を処理し、パディングとソートを行うcollate関数。
    Args:
        batch (list of tuples): Datasetの__getitem__が返す要素のリスト。
                                 各タプルは (input_ids_tensor, label_tensor)。
        padding_value (int): パディングに使用するトークンID。
    Returns:
        dict: {'input_ids': パディング・ソート済みinput_idsバッチ (tensor),
               'label': ソート済みlabelバッチ (tensor)}
    """

    # 1. input_ids と label をそれぞれのリストに分離
    input_ids_list = [item[0] for item in batch]
    labels_list = [item[1] for item in batch]

    # 2. input_ids の長さを取得し、それに基づいて事例を降順にソート
    #    (input_ids, label, length) のタプルリストを作成
    lengths = [len(ids) for ids in input_ids_list]
    # ソートキーとなる長さと共に元のデータを保持
    # (元のインデックスも保持しておくと、後でソートを戻す場合に役立つこともあるが、今回は不要)
    batch_with_lengths = sorted(zip(input_ids_list, labels_list, lengths),
                                key=lambda x: x[2],
                                reverse=True)

    # ソートされた input_ids と label を再度取り出す
    sorted_input_ids = [item[0] for item in batch_with_lengths]
    sorted_labels = [item[1] for item in batch_with_lengths]
    # sorted_lengths = [item[2] for item in batch_with_lengths] # 必要であれば長さも保持

    # 3. パディング (ソート済みの input_ids_list に対して)
    # torch.nn.utils.rnn.pad_sequence はテンソルのリストを受け取る
    # batch_first=True で出力形状が (バッチサイズ, 最大シーケンス長) になる
    input_ids_padded = pad_sequence(sorted_input_ids, batch_first=True, padding_value=padding_value)

    # 4. ラベルをテンソルにスタック (ソート済みの labels_list に対して)
    # labels_list の各要素は既にテンソル (例: tensor([0.])) なので、torch.stack でまとめる
    labels_batched = torch.stack(sorted_labels)

    return {'input_ids': input_ids_padded, 'label': labels_batched}


# --- collate関数の動作テスト ---
print("collate関数の動作テストを行います。")

# 問題文の例に近いダミーデータを作成 (問題71の train_dataset_processed の形式を模倣)
# train_dataset_processed の最初の数件を使うのがより実践的
if len(train_dataset_processed) >= 4:
    print("\n訓練データの最初の4事例を使ってテストします。")
    sample_batch_from_dataset = [train_torch_dataset_item for train_torch_dataset_item in SentimentDataset(train_dataset_processed[:4])]
    # SentimentDatasetの__getitem__は(input_ids, label)のタプルを返すので、それがリストになったものがsample_batch_from_dataset
else:
    print("\n訓練データが4件未満のため、手動でダミーデータを作成します。")
    # 手動ダミーデータ (input_ids は様々な長さのテンソル、label もテンソル)
    sample_batch_from_dataset = [
        (torch.tensor([5785, 66, 113845, 18, 12, 15095, 1594]), torch.tensor([0.])), # len 7
        (torch.tensor([3475, 87, 15888, 90, 27695, 42637]), torch.tensor([0.])),    # len 6
        (torch.tensor([4, 5053, 45, 3305, 31647, 348, 904, 2815, 47, 1276, 1964]), torch.tensor([1.])), # len 11
        (torch.tensor([987, 14528, 4941, 873, 12, 208, 898]), torch.tensor([0.]))  # len 7
    ]

# collate関数をテスト
# PAD_ID は問題70で定義されているはず (通常は0)
collated_batch = collate_fn_custom_pad_sort(sample_batch_from_dataset, padding_value=PAD_ID)

print("\ncollate関数を通した結果:")
print("input_ids (パディング・ソート済み):")
print(collated_batch['input_ids'])
print("input_ids の形状:", collated_batch['input_ids'].shape)
print("\nlabel (ソート済み):")
print(collated_batch['label'])
print("label の形状:", collated_batch['label'].shape)

# 問題文の期待する出力形式と一致しているか確認
# 期待される input_ids (長さ11が最長、降順ソート):
# tensor([[    4,  5053,    45,  3305, 31647,   348,   904,  2815,    47,  1276,  1964],
#         [ 5785,    66,113845,    18,    12, 15095,  1594,     0,     0,     0,     0],
#         [  987, 14528,  4941,   873,    12,   208,   898,     0,     0,     0,     0],
#         [ 3475,    87, 15888,    90, 27695, 42637,     0,     0,     0,     0,     0]])
# 期待される label (対応してソート):
# tensor([[1.],
#         [0.],
#         [0.],
#         [0.]])

# この collate_fn_custom_pad_sort 関数を、次の問題76で DataLoader の collate_fn として指定します。

## 76. ミニバッチ学習

問題75のパディングの処理を活用して、ミニバッチでモデルを学習せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence # collate_fn_custom_pad_sort で使用

# --- 前提となる変数・クラス・関数の定義 ---
# PAD_ID (問題70)
# embedding_matrix, vocab_size, embedding_dim (問題70)
# SimpleBOWClassifier クラス (問題72)
# model_bow (問題72でインスタンス化、問題73で一度学習させたが、再学習するので再インスタンス化も可)
# train_dataset_processed, dev_dataset_processed (問題71)
# SentimentDataset クラス (問題73または75で定義)
# collate_fn_custom_pad_sort 関数 (問題75で定義)

# --- 変数・クラスが現在のセッションに存在するか確認 ---
required_vars = ['PAD_ID', 'embedding_matrix', 'vocab_size', 'embedding_dim',
                 'model_bow', 'train_dataset_processed', 'dev_dataset_processed']
for var_name in required_vars:
    if var_name not in locals():
        print(f"エラー: 前提となる変数 '{var_name}' が定義されていません。")
        print("問題70, 71, 72, (73)を先に実行してください。")
        raise NameError(f"Variable '{var_name}' is not defined.")

required_classes_funcs = ['SentimentDataset', 'collate_fn_custom_pad_sort', 'SimpleBOWClassifier']
for item_name in required_classes_funcs:
    if item_name not in locals():
        print(f"エラー: 前提となるクラス/関数 '{item_name}' が定義されていません。")
        print("問題72, 73, 75のコードを確認・実行してください。")
        raise NameError(f"Class/Function '{item_name}' is not defined.")

# --- ここから問題76の処理 ---

# モデルを再初期化 (問題73で学習した重みをリセットして、ミニバッチ学習の効果を新たに見るため)
# もし問題73の学習結果を引き継ぎたい場合は、この行はコメントアウト
print("モデルの重みを再初期化します...")
embedding_tensor = torch.tensor(embedding_matrix, dtype=torch.float) # 問題72と同様
model_bow = SimpleBOWClassifier(vocab_size, embedding_dim, embedding_tensor, PAD_ID)


# データローダーの準備 (ミニバッチ化とカスタムcollate_fnを使用)
batch_size = 32 # ミニバッチのサイズ (64, 128なども試せる)

train_torch_dataset_mb = SentimentDataset(train_dataset_processed)
train_dataloader_mb = DataLoader(train_torch_dataset_mb,
                                 batch_size=batch_size,
                                 shuffle=True,
                                 collate_fn=lambda b: collate_fn_custom_pad_sort(b, padding_value=PAD_ID))

if dev_dataset_processed:
    dev_torch_dataset_mb = SentimentDataset(dev_dataset_processed)
    dev_dataloader_mb = DataLoader(dev_torch_dataset_mb,
                                   batch_size=batch_size,
                                   shuffle=False, # 検証時はシャッフル不要
                                   collate_fn=lambda b: collate_fn_custom_pad_sort(b, padding_value=PAD_ID))
else:
    dev_dataloader_mb = None


# 学習パラメータ
learning_rate_mb = 1e-3 # 問題73と同じか、調整しても良い
num_epochs_mb = 5     # 学習エポック数 (適宜調整)

# 損失関数と最適化アルゴリズム
criterion_mb = nn.BCEWithLogitsLoss()
optimizer_mb = optim.Adam(model_bow.parameters(), lr=learning_rate_mb)

print(f"\nミニバッチ学習 (バッチサイズ={batch_size}) を開始します...")
# --- 学習ループ ---
for epoch in range(num_epochs_mb):
    model_bow.train() # モデルを学習モードに
    running_loss = 0.0
    num_processed_samples = 0

    for i, batch_data in enumerate(train_dataloader_mb):
        input_ids_batch = batch_data['input_ids'] # collate_fnの返り値に合わせてアクセス
        labels_batch = batch_data['label']

        optimizer_mb.zero_grad()
        outputs = model_bow(input_ids_batch)
        loss = criterion_mb(outputs, labels_batch)
        loss.backward()
        optimizer_mb.step()

        running_loss += loss.item() * input_ids_batch.size(0)
        num_processed_samples += input_ids_batch.size(0)

        if (i + 1) % (len(train_dataloader_mb) // 5) == 0 or (i + 1) == len(train_dataloader_mb): # 約20%ごとと最後に表示
            avg_loss_so_far = running_loss / num_processed_samples
            print(f"  Epoch [{epoch+1}/{num_epochs_mb}], Batch [{i+1}/{len(train_dataloader_mb)}], Avg Loss: {avg_loss_so_far:.4f}")

    epoch_loss = running_loss / len(train_torch_dataset_mb)
    print(f"Epoch [{epoch+1}/{num_epochs_mb}] 完了, 平均訓練損失: {epoch_loss:.4f}")

    # 各エポックの終わりに検証データで性能評価
    if dev_dataloader_mb:
        model_bow.eval() # モデルを評価モードに
        correct_dev = 0
        total_dev = 0
        dev_loss = 0.0
        with torch.no_grad():
            for batch_data_dev in dev_dataloader_mb:
                input_ids_dev = batch_data_dev['input_ids']
                labels_dev = batch_data_dev['label']

                outputs_dev = model_bow(input_ids_dev)
                loss_dev_batch = criterion_mb(outputs_dev, labels_dev)
                dev_loss += loss_dev_batch.item() * input_ids_dev.size(0)

                predicted_probs = torch.sigmoid(outputs_dev)
                predicted_labels = (predicted_probs > 0.5).float()
                total_dev += labels_dev.size(0)
                correct_dev += (predicted_labels == labels_dev).sum().item()

        avg_dev_loss = dev_loss / total_dev if total_dev > 0 else 0
        dev_accuracy = 100 * correct_dev / total_dev if total_dev > 0 else 0
        print(f"  Epoch [{epoch+1}/{num_epochs_mb}], 検証データ: 平均損失={avg_dev_loss:.4f}, 正解率={dev_accuracy:.2f}%")

print("\nミニバッチ学習が完了しました。")

# --- 学習したモデルの開発セットにおける最終的な正解率を求める ---
if dev_dataloader_mb:
    print("\n--- 最終評価 (開発セット) ---")
    model_bow.eval()
    final_correct_dev = 0
    final_total_dev = 0
    with torch.no_grad():
        for batch_data_dev in dev_dataloader_mb:
            input_ids_dev = batch_data_dev['input_ids']
            labels_dev = batch_data_dev['label']
            outputs_dev = model_bow(input_ids_dev)
            predicted_probs = torch.sigmoid(outputs_dev)
            predicted_labels = (predicted_probs > 0.5).float()
            final_total_dev += labels_dev.size(0)
            final_correct_dev += (predicted_labels == labels_dev).sum().item()

    if final_total_dev > 0:
        final_dev_accuracy = 100 * final_correct_dev / final_total_dev
        print(f"開発セットにおける最終正解率: {final_dev_accuracy:.2f}%")
        print(f"  正解した事例数: {final_correct_dev}")
        print(f"  総事例数: {final_total_dev}")
    else:
        print("開発データでの評価ができませんでした。")
else:
    print("開発データローダーが準備されていないため、最終評価をスキップします。")

## 77. GPU上での学習

問題76のモデル学習をGPU上で実行せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence # collate_fn_custom_pad_sort で使用
import time # 学習時間の計測用

# --- 前提となる変数・クラス・関数の定義 (問題76と同様) ---
# PAD_ID, embedding_matrix, vocab_size, embedding_dim
# SimpleBOWClassifier クラス
# train_dataset_processed, dev_dataset_processed
# SentimentDataset クラス
# collate_fn_custom_pad_sort 関数

# --- 変数・クラスが現在のセッションに存在するか確認 ---
required_vars_p77 = ['PAD_ID', 'embedding_matrix', 'vocab_size', 'embedding_dim',
                     'train_dataset_processed', 'dev_dataset_processed']
for var_name in required_vars_p77:
    if var_name not in locals():
        print(f"エラー: 前提となる変数 '{var_name}' が定義されていません。")
        print("問題70, 71を先に実行してください。")
        raise NameError(f"Variable '{var_name}' is not defined.")

required_classes_funcs_p77 = ['SentimentDataset', 'collate_fn_custom_pad_sort', 'SimpleBOWClassifier']
for item_name in required_classes_funcs_p77:
    if item_name not in locals():
        print(f"エラー: 前提となるクラス/関数 '{item_name}' が定義されていません。")
        print("問題72, 73, 75のコードを確認・実行してください。")
        raise NameError(f"Class/Function '{item_name}' is not defined.")

# --- ここから問題77の処理 ---

# 1. GPUの利用可能性確認とデバイス設定
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU ({torch.cuda.get_device_name(0)}) を使用します。")
else:
    device = torch.device("cpu")
    print("GPUが利用できません。CPUを使用します。")

# モデルを再初期化し、指定デバイスへ転送
print("\nモデルを再初期化し、デバイスに転送します...")
embedding_tensor_p77 = torch.tensor(embedding_matrix, dtype=torch.float)
model_bow_gpu = SimpleBOWClassifier(vocab_size, embedding_dim, embedding_tensor_p77, PAD_ID)
model_bow_gpu.to(device) # ★★★ モデルをデバイスへ ★★★

# データローダーの準備 (問題76と同様)
batch_size_gpu = 32

train_torch_dataset_gpu = SentimentDataset(train_dataset_processed)
train_dataloader_gpu = DataLoader(train_torch_dataset_gpu,
                                  batch_size=batch_size_gpu,
                                  shuffle=True,
                                  collate_fn=lambda b: collate_fn_custom_pad_sort(b, padding_value=PAD_ID))

if dev_dataset_processed:
    dev_torch_dataset_gpu = SentimentDataset(dev_dataset_processed)
    dev_dataloader_gpu = DataLoader(dev_torch_dataset_gpu,
                                    batch_size=batch_size_gpu,
                                    shuffle=False,
                                    collate_fn=lambda b: collate_fn_custom_pad_sort(b, padding_value=PAD_ID))
else:
    dev_dataloader_gpu = None

# 学習パラメータ
learning_rate_gpu = 1e-3
num_epochs_gpu = 5 # 問題76と同じエポック数で比較

# 損失関数と最適化アルゴリズム
criterion_gpu = nn.BCEWithLogitsLoss()
optimizer_gpu = optim.Adam(model_bow_gpu.parameters(), lr=learning_rate_gpu)

print(f"\nGPU上でのミニバッチ学習 (バッチサイズ={batch_size_gpu}) を開始します...")
start_time = time.time()
# --- 学習ループ ---
for epoch in range(num_epochs_gpu):
    model_bow_gpu.train()
    running_loss = 0.0
    num_processed_samples = 0

    for i, batch_data in enumerate(train_dataloader_gpu):
        # ★★★ データをデバイスへ転送 ★★★
        input_ids_batch = batch_data['input_ids'].to(device)
        labels_batch = batch_data['label'].to(device)

        optimizer_gpu.zero_grad()
        outputs = model_bow_gpu(input_ids_batch)
        loss = criterion_gpu(outputs, labels_batch)
        loss.backward()
        optimizer_gpu.step()

        running_loss += loss.item() * input_ids_batch.size(0)
        num_processed_samples += input_ids_batch.size(0)

        if (i + 1) % (len(train_dataloader_gpu) // 5) == 0 or (i + 1) == len(train_dataloader_gpu):
            avg_loss_so_far = running_loss / num_processed_samples
            print(f"  Epoch [{epoch+1}/{num_epochs_gpu}], Batch [{i+1}/{len(train_dataloader_gpu)}], Avg Loss: {avg_loss_so_far:.4f}")

    epoch_loss = running_loss / len(train_torch_dataset_gpu)
    print(f"Epoch [{epoch+1}/{num_epochs_gpu}] 完了, 平均訓練損失: {epoch_loss:.4f}")

    if dev_dataloader_gpu:
        model_bow_gpu.eval()
        correct_dev = 0
        total_dev = 0
        dev_loss = 0.0
        with torch.no_grad():
            for batch_data_dev in dev_dataloader_gpu:
                # ★★★ データをデバイスへ転送 ★★★
                input_ids_dev = batch_data_dev['input_ids'].to(device)
                labels_dev = batch_data_dev['label'].to(device)

                outputs_dev = model_bow_gpu(input_ids_dev)
                loss_dev_batch = criterion_gpu(outputs_dev, labels_dev) # 損失計算もデバイス上
                dev_loss += loss_dev_batch.item() * input_ids_dev.size(0)

                predicted_probs = torch.sigmoid(outputs_dev)
                predicted_labels = (predicted_probs > 0.5).float()
                total_dev += labels_dev.size(0)
                correct_dev += (predicted_labels == labels_dev).sum().item()

        avg_dev_loss = dev_loss / total_dev if total_dev > 0 else 0
        dev_accuracy = 100 * correct_dev / total_dev if total_dev > 0 else 0
        print(f"  Epoch [{epoch+1}/{num_epochs_gpu}], 検証データ: 平均損失={avg_dev_loss:.4f}, 正解率={dev_accuracy:.2f}%")

end_time = time.time()
print(f"\nGPU上でのミニバッチ学習が完了しました。所要時間: {end_time - start_time:.2f} 秒")

# --- 学習したモデルの開発セットにおける最終的な正解率を求める ---
if dev_dataloader_gpu:
    print("\n--- 最終評価 (開発セット) ---")
    model_bow_gpu.eval()
    final_correct_dev = 0
    final_total_dev = 0
    with torch.no_grad():
        for batch_data_dev in dev_dataloader_gpu:
            # ★★★ データをデバイスへ転送 ★★★
            input_ids_dev = batch_data_dev['input_ids'].to(device)
            labels_dev = batch_data_dev['label'].to(device)

            outputs_dev = model_bow_gpu(input_ids_dev)
            predicted_probs = torch.sigmoid(outputs_dev)
            predicted_labels = (predicted_probs > 0.5).float()
            final_total_dev += labels_dev.size(0)
            final_correct_dev += (predicted_labels == labels_dev).sum().item()

    if final_total_dev > 0:
        final_dev_accuracy_gpu = 100 * final_correct_dev / final_total_dev
        print(f"開発セットにおける最終正解率 (GPU学習): {final_dev_accuracy_gpu:.2f}%")
        print(f"  正解した事例数: {final_correct_dev}")
        print(f"  総事例数: {final_total_dev}")
    else:
        print("開発データでの評価ができませんでした。")
else:
    print("開発データローダーが準備されていないため、最終評価をスキップします。")

## 78. 単語埋め込みのファインチューニング

問題77の学習において、単語埋め込みのパラメータも同時に更新するファインチューニングを導入せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
import torch
torch.cuda.empty_cache()

In [ ]:
import numpy as np
from gensim.models import KeyedVectors
import pandas as pd # SST-2データセットを読み込むために追加
import os # ファイルパス処理のために追加

# ... (前のコード: drive.mount, %cd, uv pip install など) ...

# 問題60でSST-2データを展開した親ディレクトリへのパスを想定
base_data_dir_ch7 = '../data/SST-2_data'
train_file_path = os.path.join(base_data_dir_ch7, "SST-2/train.tsv")
dev_file_path = os.path.join(base_data_dir_ch7, "SST-2/dev.tsv")

# Google News単語ベクトルモデルのパス
model_path = '../data/GoogleNews-vectors-negative300.bin.gz'

# モデルの読み込み (時間がかかることがあります)
print("単語ベクトルモデルを読み込んでいます... (数分かかる場合があります)")
try:
    word_vectors = KeyedVectors.load_word2vec_format(model_path, binary=True)
    print("モデルの読み込みが完了しました。")
except FileNotFoundError:
    print(f"エラー: 指定されたパスにファイルが見つかりません: {model_path}")
    raise FileNotFoundError("Google Newsの単語ベクトルファイルをダウンロードし、正しいパスを指定してください。")
except Exception as e:
    print(f"モデルの読み込み中またはベクトル取得中にエラーが発生しました: {e}")
    raise e

# --- SST-2データセットから語彙を収集 ---
print("\nSST-2データセットの語彙を収集しています...")
sst2_vocab = set()
try:
    train_df = pd.read_csv(train_file_path, sep='\t')
    dev_df = pd.read_csv(dev_file_path, sep='\t')

    # 訓練セットと開発セットの全ての単語を収集
    for text in train_df['sentence']:
        sst2_vocab.update(str(text).split(' '))
    for text in dev_df['sentence']:
        sst2_vocab.update(str(text).split(' '))

    print(f"SST-2データセットに出現するユニークな単語数: {len(sst2_vocab)}")

except FileNotFoundError:
    print("エラー: SST-2データセットファイルが見つかりません。問題60でダウンロードしてください。")
    raise FileNotFoundError("SST-2 dataset files not found.")
except KeyError:
    print("エラー: SST-2データセットファイルに必要な列('sentence'または'label')が見つかりません。")
    raise KeyError("Required columns not found in SST-2 dataset.")
except Exception as e:
    print(f"SST-2データセットの語彙収集中にエラーが発生しました: {e}")
    raise e

# --- Google Newsの語彙に含まれるSST-2の単語とPADトークンで新しい語彙を作成 ---
PAD_TOKEN = "<PAD>"
PAD_ID = 0
# <PAD>トークンを最初に追加
word_to_id = {PAD_TOKEN: PAD_ID}
id_to_word = {PAD_ID: PAD_TOKEN}

# Google Newsの語彙に含まれるSST-2の単語を新しい語彙に追加
current_id = 1
# sorted() で単語をソートすると、IDが単語のアルファベット順になる
# ソートしない場合は、元のword_vectorsの順序になる (再現性にはソートが推奨)
# for word in sorted(list(sst2_vocab)): # SST2の単語をソートして追加する場合
#     if word in word_vectors:
#         word_to_id[word] = current_id
#         id_to_word[current_id] = word
#         current_id += 1

# または、Google Newsの語彙順序でSST2の単語を追加する場合
# (こちらの方が元の埋め込み行列の構造に近いかもしれない)
for word in word_vectors.index_to_key:
    if word in sst2_vocab:
         word_to_id[word] = current_id
         id_to_word[current_id] = word
         current_id += 1


# 新しい語彙サイズと埋め込み次元数を定義
vocab_size = len(word_to_id)
embedding_dim = word_vectors.vector_size # 300次元のはず

print(f"\n新しい語彙サイズ (|V|): {vocab_size}")
print(f"単語ベクトルの次元数 (d_emb): {embedding_dim}")

# 新しい単語埋め込み行列Eを初期化 (0行目はパディング用にゼロベクトル)
# サイズは新しい語彙数 x 埋め込み次元
embedding_matrix = np.zeros((vocab_size, embedding_dim), dtype=np.float32)

print(f"新しい単語埋め込み行列 E の形状: {embedding_matrix.shape}")

# 新しい語彙に対応する単語ベクトルを embedding_matrix にコピー
# ID=0 は <PAD> なので、実際の単語は ID=1 から開始
for word, token_id in word_to_id.items():
    if word != PAD_TOKEN:
        # word_vectorsに単語が存在するかは既にチェック済みだが念のため
        if word in word_vectors:
             embedding_matrix[token_id] = word_vectors[word]
        # else: これは発生しないはず (sst2_vocab & word_vectors.index_to_key の積集合を取っているため)


print("\n単語埋め込み行列とIDマッピングの構築が完了しました (SST-2語彙に限定)。")


## 79. アーキテクチャの変更

ニューラルネットワークのアーキテクチャを自由に変更し、モデルを学習せよ。また、学習したモデルの開発セットにおける正解率を求めよ。例えば、テキストの特徴ベクトル（単語埋め込みの平均ベクトル）に対して多層のニューラルネットワークを通したり、畳み込みニューラルネットワーク（CNN; Convolutional Neural Network）や再帰型ニューラルネットワーク（RNN; Recurrent Neural Network）などのモデルの学習に挑戦するとよい。

In [ ]:
# Ensure this uses the word_to_id and vocab_size from cell 15
print("Re-processing datasets with the REDUCED vocabulary from cell 15...")

# Use the word_to_id map created in cell 15
# This word_to_id from cell 15 IS the reduced one.
train_dataset_processed_reduced = load_and_process_sst2_dataset(train_file_path, word_to_id, "訓練データ (Reduced Vocab)")
dev_dataset_processed_reduced = load_and_process_sst2_dataset(dev_file_path, word_to_id, "開発データ (Reduced Vocab)")

# Update the variables that will be used by the DataLoader in cell 18
train_dataset_processed = train_dataset_processed_reduced
dev_dataset_processed = dev_dataset_processed_reduced

In [ ]:
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import time
import numpy as np

# --- 前提となる変数・クラス・関数の定義 (問題78と同様) ---
# PAD_ID, embedding_matrix, vocab_size, embedding_dim
# train_dataset_processed, dev_dataset_processed
# SentimentDataset クラス
# collate_fn_custom_pad_sort 関数

# --- 変数・クラスが現在のセッションに存在するか確認 ---
required_vars_p79 = ['PAD_ID', 'embedding_matrix', 'vocab_size', 'embedding_dim',
                     'train_dataset_processed', 'dev_dataset_processed']
for var_name in required_vars_p79:
    if var_name not in locals():
        print(f"エラー: 前提となる変数 '{var_name}' が定義されていません。")
        raise NameError(f"Variable '{var_name}' is not defined.")

required_classes_funcs_p79 = ['SentimentDataset', 'collate_fn_custom_pad_sort']
for item_name in required_classes_funcs_p79:
    if item_name not in locals():
        print(f"エラー: 前提となるクラス/関数 '{item_name}' が定義されていません。")
        raise NameError(f"Class/Function '{item_name}' is not defined.")

# --- ここから問題79の処理 ---

# 1. MLPモデルクラスの定義
class MLPClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim,
                 pretrained_embeddings, padding_idx, dropout_rate=0.5, fine_tune_embeddings=True):
        super(MLPClassifier, self).__init__()

        # 既存の埋め込み層の定義をコメントアウトまたは削除
        # self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        # if fine_tune_embeddings:
        #     self.embedding.weight = nn.Parameter(pretrained_embeddings, requires_grad=True)
        # else:
        #     self.embedding.weight = nn.Parameter(pretrained_embeddings, requires_grad=False)

        # nn.Embedding.from_pretrained を使用して書き換え
        # pretrained_embeddings は torch.FloatTensor である必要があります。
        # (embedding_tensor_p79 = torch.tensor(embedding_matrix, dtype=torch.float) で既にそうなっているはずです)
        self.embedding = nn.Embedding.from_pretrained(
            embeddings=pretrained_embeddings,
            freeze=(not fine_tune_embeddings),  # freeze=True は勾配計算しない (requires_grad=False と同等)
            padding_idx=padding_idx
        )
        # nn.Embedding.from_pretrained は padding_idx の行を自動的にゼロでマスクします。
        # 元の embedding_matrix[PAD_ID] がゼロベクトルであれば問題ありません。

        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, input_ids):
        # (フォワードパスの残りは変更なし)
        # ...
        embedded = self.embedding(input_ids)

        mask = (input_ids != self.embedding.padding_idx).unsqueeze(-1).float()
        lengths = mask.sum(dim=1).clamp(min=1)
        sum_embedded = (embedded * mask).sum(dim=1)
        mean_embedded = sum_embedded / lengths

        if torch.isnan(mean_embedded).any() or torch.isinf(mean_embedded).any():
            # エラーハンドリングやデバッグ情報をここに追加できます
            print("NaN or Inf found in mean_embedded after embedding layer!")
            # print("Input IDs causing issue:", input_ids)
            # print("Lengths:", lengths)
            # print("Sum embedded:", sum_embedded)
            # このような状況が発生した場合、さらなる調査が必要です。

        out = self.fc1(mean_embedded)
        out = self.relu(out)
        out = self.dropout(out)
        logits = self.fc2(out)

        return logits

# --- モデルのパラメータ設定 ---
# embedding_dim は問題70から (例: 300)
hidden_dim = 128  # MLPの隠れ層の次元数 (自由に設定)
output_dim = 1    # ポジネガの2値分類なので出力は1 (ロジット)
dropout_p = 0.5   # ドロップアウト率 (過学習抑制のため)
fine_tune_emb = True # 単語埋め込みをファインチューニングするかどうか (True or False)

# デバイス設定 (問題77, 78と同様)
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU ({torch.cuda.get_device_name(0)}) を使用します。")
else:
    device = torch.device("cpu")
    print("GPUが利用できません。CPUを使用します。")

# モデルを初期化し、指定デバイスへ転送
print("\nMLPモデルを初期化し、デバイスに転送します...")
embedding_tensor_p79 = torch.tensor(embedding_matrix, dtype=torch.float)
mlp_model = MLPClassifier(vocab_size, embedding_dim, hidden_dim, output_dim,
                          embedding_tensor_p79, PAD_ID, dropout_rate=dropout_p,
                          fine_tune_embeddings=fine_tune_emb)
mlp_model.to(device)
print("設計したMLPモデルの構造:")
print(mlp_model)


# データローダーの準備 (問題76, 78と同様)
batch_size_mlp = 16
train_torch_dataset_mlp = SentimentDataset(train_dataset_processed)
train_dataloader_mlp = DataLoader(train_torch_dataset_mlp,
                                 batch_size=batch_size_mlp,
                                 shuffle=True,
                                 collate_fn=lambda b: collate_fn_custom_pad_sort(b, padding_value=PAD_ID))

if dev_dataset_processed:
    dev_torch_dataset_mlp = SentimentDataset(dev_dataset_processed)
    dev_dataloader_mlp = DataLoader(dev_torch_dataset_mlp,
                                   batch_size=batch_size_mlp,
                                   shuffle=False,
                                   collate_fn=lambda b: collate_fn_custom_pad_sort(b, padding_value=PAD_ID))
else:
    dev_dataloader_mlp = None

# 学習パラメータ
learning_rate_mlp = 1e-4 # ファインチューニング時は小さめが安定しやすい
num_epochs_mlp = 10      # エポック数を増やして学習効果を見る (適宜調整)

# 損失関数と最適化アルゴリズム
criterion_mlp = nn.BCEWithLogitsLoss()
optimizer_mlp = optim.Adam(mlp_model.parameters(), lr=learning_rate_mlp)

print(f"\nMLPモデルの学習 (バッチサイズ={batch_size_mlp}, 埋め込みファインチューニング={fine_tune_emb}) を開始します...")
start_time_mlp = time.time()
# --- 学習ループ (問題77, 78とほぼ同じ) ---
for epoch in range(num_epochs_mlp):
    mlp_model.train() # モデルを学習モードに
    running_loss = 0.0
    num_processed_samples = 0

    for i, batch_data in enumerate(train_dataloader_mlp):
        input_ids_batch = batch_data['input_ids'].to(device)
        labels_batch = batch_data['label'].to(device)

        optimizer_mlp.zero_grad()
        outputs = mlp_model(input_ids_batch) # モデルのフォワードパス
        loss = criterion_mlp(outputs, labels_batch)
        loss.backward()
        optimizer_mlp.step()

        running_loss += loss.item() * input_ids_batch.size(0)
        num_processed_samples += input_ids_batch.size(0)

        if (i + 1) % (len(train_dataloader_mlp) // 5) == 0 or (i + 1) == len(train_dataloader_mlp):
            avg_loss_so_far = running_loss / num_processed_samples
            print(f"  Epoch [{epoch+1}/{num_epochs_mlp}], Batch [{i+1}/{len(train_dataloader_mlp)}], Avg Loss: {avg_loss_so_far:.4f}")

    epoch_loss = running_loss / len(train_torch_dataset_mlp)
    print(f"Epoch [{epoch+1}/{num_epochs_mlp}] 完了, 平均訓練損失: {epoch_loss:.4f}")

    if dev_dataloader_mlp:
        mlp_model.eval() # モデルを評価モードに
        correct_dev = 0
        total_dev = 0
        dev_loss = 0.0
        with torch.no_grad():
            for batch_data_dev in dev_dataloader_mlp:
                input_ids_dev = batch_data_dev['input_ids'].to(device)
                labels_dev = batch_data_dev['label'].to(device)

                outputs_dev = mlp_model(input_ids_dev)
                loss_dev_batch = criterion_mlp(outputs_dev, labels_dev)
                dev_loss += loss_dev_batch.item() * input_ids_dev.size(0)

                predicted_probs = torch.sigmoid(outputs_dev)
                predicted_labels = (predicted_probs > 0.5).float()
                total_dev += labels_dev.size(0)
                correct_dev += (predicted_labels == labels_dev).sum().item()

        avg_dev_loss = dev_loss / total_dev if total_dev > 0 else 0
        dev_accuracy = 100 * correct_dev / total_dev if total_dev > 0 else 0
        print(f"  Epoch [{epoch+1}/{num_epochs_mlp}], 検証データ: 平均損失={avg_dev_loss:.4f}, 正解率={dev_accuracy:.2f}%")

end_time_mlp = time.time()
print(f"\nMLPモデルの学習が完了しました。所要時間: {end_time_mlp - start_time_mlp:.2f} 秒")

# --- 学習したモデルの開発セットにおける最終的な正解率を求める ---
if dev_dataloader_mlp:
    print("\n--- 最終評価 (開発セット、MLPモデル) ---")
    mlp_model.eval()
    final_correct_dev_mlp = 0
    final_total_dev_mlp = 0
    with torch.no_grad():
        for batch_data_dev in dev_dataloader_mlp:
            input_ids_dev = batch_data_dev['input_ids'].to(device)
            labels_dev = batch_data_dev['label'].to(device)

            outputs_dev = mlp_model(input_ids_dev)
            predicted_probs = torch.sigmoid(outputs_dev)
            predicted_labels = (predicted_probs > 0.5).float()
            final_total_dev_mlp += labels_dev.size(0)
            final_correct_dev_mlp += (predicted_labels == labels_dev).sum().item()

    if final_total_dev_mlp > 0:
        final_dev_accuracy_mlp = 100 * final_correct_dev_mlp / final_total_dev_mlp
        print(f"開発セットにおける最終正解率 (MLPモデル): {final_dev_accuracy_mlp:.2f}%")
        print(f"  正解した事例数: {final_correct_dev_mlp}")
        print(f"  総事例数: {final_total_dev_mlp}")
    else:
        print("開発データでの評価ができませんでした。")
else:
    print("開発データローダーが準備されていないため、最終評価をスキップします。")